In [1]:
import logging
logging.basicConfig(level=logging.ERROR)


## Introducción: productores y consumidores usando tópicos

In [1]:
from kafka import KafkaProducer
SERVER  = "kafka_b:9094"
producer = KafkaProducer(bootstrap_servers=SERVER)

Enviemos un mensaje al tópico website_events:

In [18]:
future = producer.send(
    "website_events", 
    b"{'type': 'click', 'location': '/index.html'}"
)
# Alternativamente: "{'type': 'view', 'location': '/index.html'}".encode("utf-8")

future

¡Devuelve un futuro! Ni el envío es bloqueante para el productor. 
Inspeccionemos un rato más tarde la información generada:

In [20]:
future.get()

RecordMetadata(topic='website_events', partition=0, topic_partition=TopicPartition(topic='website_events', partition=0), offset=65, timestamp=1719276316091, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=44, serialized_header_size=-1)

Enviemos cinco mensajes:


In [6]:
import json

json_encoder = json.JSONEncoder()

for _ in range(5):
    producer.send(
        "website_events", 
        json_encoder.encode(
            dict(type="click", location="/index.html")
        ).encode("utf-8")
    )

## Envío usando particiones

In [2]:
def partitioner(key_bytes, all_partitions, available_partitions):
    partition = all_partitions[int(key_bytes.decode("utf-8"))]
    print(f"From <{key_bytes}> inferred <{partition}>")
    print(f"From all <{all_partitions}> and available <{available_partitions}>")
    return partition

In [3]:
partition_producer = KafkaProducer(
    bootstrap_servers=SERVER,
    partitioner=partitioner,
)

Enviemos datos a particiones explícitamente, para entender el manejo de particiones:

In [19]:
partition_producer.send(
    "metrics",
    key=b'0',
    value=b"hola"
)

From <b'0'> inferred <0>
From all <[0, 1]> and available <[0, 1]>


In [30]:
partition_producer.send(
    "metrics",
    key=b'1',
    value=b"mundo"
)

From <b'1'> inferred <1>
From all <[0, 1]> and available <[0, 1]>
